# Getting Started

## Setup Database Environment - Retail CLV Regression Demo

This Notebook is used to setup the required database objects including mock retail data for a **Customer Lifetime Value (CLV) Regression** demo.

### Tables Created:
1. **CUSTOMERS**: Customer profile features (4 features)
2. **PURCHASE_BEHAVIOR**: Transaction-based features (3 features + target)

The tables join on `CUSTOMER_ID` to demonstrate feature engineering workflows.

In [ ]:
%load_ext autoreload
%autoreload 2

**Install SQLGlot** <br>
Install SQLGlot with pip install in the conda environment **py-snowpark_df_ml_fs** by running the following command in the same terminal window.  We will use this package to format the SQL produced from Snowpark so that it is human-readable in the Dynamic Tables that Feature Store creates.  Installing within the Notebook, as other users have reported issues trying to install directly within the OS.

In [ ]:
!python3 -m pip install "sqlglot[rs]" --no-deps

#### Notebook Packages

In [ ]:
# Python packages
import os
from os import listdir
from os.path import isfile, join
import time
import json
import datetime


# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.version import VERSION
from snowflake.ml.utils import connection_params

from helper.useful_fns import run_sql

### Setup Snowflake connection and database parameters

Change the settings below if you want to if need to apply to your Snowflake Account.

E.g. if you need to use a different role with ACCOUNTADMIN privileges to setup the environment

In [ ]:
# ===========================================
# CONFIGURATION - Modify these as needed
# ===========================================

# Roles
admin_role = 'ACCOUNTADMIN'              # Role with ACCOUNTADMIN privileges for setup
demo_role = 'RETAIL_REGRESSION_DEMO_ROLE'           # The data scientist role for this demo

# Database
database_name = 'RETAIL_REGRESSION_DEMO'

# Schema
schema_name = 'DS'

# Warehouse
warehouse_name = 'RETAIL_REGRESSION_DEMO_WH'
warehouse_size = 'SMALL'

# Data Generation
num_customers = 10000                     # Number of customers to generate 

In [ ]:
# Create Snowflake Session object
with open('connection.json', 'r') as f:
    connection_parameters = json.load(f)
# connection_parameters = connection_params.SnowflakeLoginOptions("ak32940")
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

In [ ]:
# Current Environment Details
print('\nConnection Established with the following parameters:')
print(f'User                        : {snowflake_environment[0][0]}')
print(f'Role                        : {session.get_current_role()}')
print(f'Database                    : {session.get_current_database()}')
print(f'Schema                      : {session.get_current_schema()}')
print(f'Warehouse                   : {session.get_current_warehouse()}')
print(f'Snowflake version           : {snowflake_environment[0][1]}')
print(f'Snowpark for Python version : {snowpark_version[0]}.{snowpark_version[1]}.{snowpark_version[2]} \n')

In [ ]:
run_sql(f'''use role {admin_role}''', session)

In [ ]:
# Create compute pool
def create_compute_pool(name: str, instance_family: str, min_nodes: int = 1, max_nodes: int = 10):
    query = f"""
        CREATE COMPUTE POOL IF NOT EXISTS {name}
            MIN_NODES = {min_nodes}
            MAX_NODES = {max_nodes}
            INSTANCE_FAMILY = {instance_family}
    """
    return session.sql(query).collect()

compute_pool = "CLV_MODEL_POOL_CPU"
create_compute_pool(compute_pool, "CPU_X64_L")

In [ ]:
# Setup master role and permissions
run_sql(f'''use role {admin_role}''', session)

# Create the demo role
run_sql(f'''create role if not exists {demo_role}''', session)

# Grant role to SYSADMIN (best practice for role hierarchy)
run_sql(f'''grant role {demo_role} to role SYSADMIN''', session)

# Create warehouse
run_sql(f'''create warehouse if not exists {warehouse_name} 
            warehouse_size = {warehouse_size}
            auto_suspend = 60
            auto_resume = true
            initially_suspended = true''', session)

# Grant warehouse permissions to demo role
run_sql(f'''grant all on warehouse {warehouse_name} to role {demo_role}''', session)

# Use the warehouse
run_sql(f'''use warehouse {warehouse_name}''', session)

# Grant task execution permissions to demo role
run_sql(f'''grant execute managed task on account to role {demo_role}''', session)
run_sql(f'''grant execute task on account to role {demo_role}''', session)


In [ ]:
# Database setup
run_sql(f'''use role {admin_role}''', session)

# Create database
# run_sql(f'''create database if not exists {database_name}''', session)
run_sql(f'''create or replace database {database_name}''', session)

# Grant database permissions to demo role
run_sql(f'''grant all on database {database_name} to role {demo_role}''', session)
run_sql(f'''grant all on all schemas in database {database_name} to role {demo_role}''', session)
run_sql(f'''grant all on future schemas in database {database_name} to role {demo_role}''', session)


In [ ]:
# Schema setup with permissions
# Switch to demo role for schema creation (to ensure ownership)
run_sql(f'''use role {demo_role}''', session)
run_sql(f'''use warehouse {warehouse_name}''', session)
run_sql(f'''use database {database_name}''', session)

# # Create schema
run_sql(f'''create schema if not exists {schema_name}''', session)
run_sql(f'''create or replace stage {schema_name}.MODEL_STAGE
    DIRECTORY = (ENABLE = TRUE)
    COMMENT = "Stage for storing regression models"
''', session)

# Switch back to admin to grant permissions
run_sql(f'''use role {admin_role}''', session)
run_sql(f'''grant create schema on database {database_name} to role {demo_role}''', session)

# Grant schema permissions to demo role
run_sql(f'''grant usage on schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant create table on schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant create view on schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant create tag on schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant create dynamic table on schema {database_name}.{schema_name} to role {demo_role}''', session)

# Grant permissions on existing and future objects
run_sql(f'''grant select, insert, update, delete on all tables in schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant select, insert, update, delete on future tables in schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant select, references on all views in schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant select, references on future views in schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant select, monitor on all dynamic tables in schema {database_name}.{schema_name} to role {demo_role}''', session)
run_sql(f'''grant select, monitor on future dynamic tables in schema {database_name}.{schema_name} to role {demo_role}''', session)


In [ ]:
# =====================================================
# TABLE 1: CUSTOMERS
# Features:
#   1. AGE: Customer age (18-75)
#   2. ANNUAL_INCOME: Estimated annual income ($20k-$200k)
#   3. LOYALTY_TIER: Customer segment (low, medium, high)
#   4. GENDER: Customer gender (male, female)
#   5. STATE: Australian state/territory (NSW, VIC, QLD, WA, SA, TAS, NT, ACT)
#   6. TENURE_MONTHS: How long they've been a customer (1-120 months)
# =====================================================

# Switch to demo role for table creation
run_sql(f'''use role {demo_role}''', session)
run_sql(f'''use warehouse {warehouse_name}''', session)
run_sql(f'''use database {database_name}''', session)
run_sql(f'''use schema {schema_name}''', session)

# Create CUSTOMERS table
run_sql(f'''
CREATE OR REPLACE TABLE CUSTOMERS (
    CUSTOMER_ID         INTEGER         NOT NULL PRIMARY KEY,
    AGE                 INTEGER         NOT NULL,
    ANNUAL_INCOME       DECIMAL(10,2)   NOT NULL,
    LOYALTY_TIER        VARCHAR         NOT NULL,
    GENDER              VARCHAR         NOT NULL,
    STATE               VARCHAR         NOT NULL,
    TENURE_MONTHS       INTEGER         NOT NULL,
    SIGNUP_DATE         DATE            NOT NULL,
    UPDATED_AT          TIMESTAMP_NTZ   DEFAULT CURRENT_TIMESTAMP()
) 
CLUSTER BY (CUSTOMER_ID)
COMMENT = 'Customer demographic features for CLV regression model'
''', session)

# Insert mock customer demographics data with categorical loyalty tiers
run_sql(f'''
INSERT INTO CUSTOMERS (
    CUSTOMER_ID,
    AGE,
    ANNUAL_INCOME,
    LOYALTY_TIER,
    GENDER,
    STATE,
    TENURE_MONTHS,
    SIGNUP_DATE,
    UPDATED_AT
)
SELECT 
    CUSTOMER_ID,
    AGE,
    ANNUAL_INCOME,
    CASE 
        WHEN tier_roll < 45 THEN 'low'
        WHEN tier_roll < 80 THEN 'medium'
        ELSE 'high'
    END AS LOYALTY_TIER,
    CASE 
        WHEN gender_roll < 50 THEN 'female'
        ELSE 'male'
    END AS GENDER,
    CASE 
        WHEN state_roll < 31 THEN 'NSW'
        WHEN state_roll < 60 THEN 'VIC'
        WHEN state_roll < 78 THEN 'QLD'
        WHEN state_roll < 87 THEN 'WA'
        WHEN state_roll < 93 THEN 'SA'
        WHEN state_roll < 97 THEN 'TAS'
        WHEN state_roll < 99 THEN 'ACT'
        ELSE 'NT'
    END AS STATE,
    TENURE_MONTHS,
    DATEADD('month', -TENURE_MONTHS, CURRENT_DATE()) AS SIGNUP_DATE,
    -- Ensure customer UPDATED_AT at least 30 days before current date
    DATEADD('day', -UNIFORM(30, 120, RANDOM()), CURRENT_TIMESTAMP()) AS UPDATED_AT
FROM (
    SELECT 
        SEQ4() + 1 AS CUSTOMER_ID,
        GREATEST(18, LEAST(75, ROUND(40 + (RANDOM() % 20) - 10 + (RANDOM() % 10)))) AS AGE,
        ROUND(CASE 
            WHEN UNIFORM(0, 100, RANDOM()) < 60 THEN UNIFORM(40000, 100000, RANDOM())
            WHEN UNIFORM(0, 100, RANDOM()) < 85 THEN UNIFORM(20000, 40000, RANDOM())
            ELSE UNIFORM(100000, 200000, RANDOM())
        END, 2) AS ANNUAL_INCOME,
        UNIFORM(0, 100, RANDOM()) AS tier_roll,
        UNIFORM(0, 100, RANDOM()) AS gender_roll,
        UNIFORM(0, 100, RANDOM()) AS state_roll,
        GREATEST(1, LEAST(120, CASE 
            WHEN UNIFORM(0, 100, RANDOM()) < 50 THEN UNIFORM(1, 24, RANDOM())
            WHEN UNIFORM(0, 100, RANDOM()) < 80 THEN UNIFORM(24, 60, RANDOM())
            ELSE UNIFORM(60, 120, RANDOM())
        END)) AS TENURE_MONTHS
    FROM TABLE(GENERATOR(ROWCOUNT => {num_customers}))
) base
''', session)

print(f'Created CUSTOMERS table with {num_customers} rows')

In [ ]:
# =====================================================
# TABLE 2: PURCHASE_BEHAVIOR
# Features:
#   1. AVG_ORDER_VALUE: Average transaction amount ($15-$500)
#   2. PURCHASE_FREQUENCY: Orders per month (0.1-8)
#   3. RETURN_RATE: Percentage of items returned (0-30%)
#   
# Target Variable:
#   4. LIFETIME_VALUE: Total customer value to predict (regression target)
# =====================================================

# Create PURCHASE_BEHAVIOR table
run_sql(f'''
CREATE OR REPLACE TABLE PURCHASE_BEHAVIOR (
    CUSTOMER_ID             INTEGER         NOT NULL PRIMARY KEY,
    AVG_ORDER_VALUE         DECIMAL(10,2)   NOT NULL,
    PURCHASE_FREQUENCY      DECIMAL(5,2)    NOT NULL,
    RETURN_RATE             DECIMAL(5,2)    NOT NULL,
    LIFETIME_VALUE          DECIMAL(12,2)   NOT NULL,
    LAST_PURCHASE_DATE      DATE            NOT NULL,
    TOTAL_ORDERS            INTEGER         NOT NULL,
    UPDATED_AT              TIMESTAMP_NTZ   DEFAULT CURRENT_TIMESTAMP(),
    FOREIGN KEY (CUSTOMER_ID) REFERENCES CUSTOMERS(CUSTOMER_ID)
)
CLUSTER BY (CUSTOMER_ID)
COMMENT = 'Purchase behavior features and CLV target for regression model'
''', session)

# Insert mock purchase behavior data with realistic correlations
run_sql(f'''
INSERT INTO PURCHASE_BEHAVIOR (
    CUSTOMER_ID, AVG_ORDER_VALUE, PURCHASE_FREQUENCY, RETURN_RATE,
    LIFETIME_VALUE, LAST_PURCHASE_DATE, TOTAL_ORDERS, UPDATED_AT
)
-- 1. BASE CUSTOMER ATTRIBUTES (string tiers mapped to numeric score)
WITH base_customers AS (
    SELECT 
        c.CUSTOMER_ID,
        c.TENURE_MONTHS,
        c.LOYALTY_TIER,
        c.ANNUAL_INCOME,
        c.SIGNUP_DATE,
        CASE 
            WHEN c.LOYALTY_TIER = 'low' THEN 1
            WHEN c.LOYALTY_TIER = 'medium' THEN 2
            WHEN c.LOYALTY_TIER = 'high' THEN 3
            ELSE 1
        END AS TIER_SCORE
    FROM CUSTOMERS c
),
    -- 2. BASE METRICS (Independent calculations, now using TIER_SCORE)
    base_metrics AS (
        SELECT 
            bc.CUSTOMER_ID,
            bc.TENURE_MONTHS,
            bc.LOYALTY_TIER,
            bc.ANNUAL_INCOME,
            bc.SIGNUP_DATE,
            bc.TIER_SCORE,
            -- Introduce occasional tier inversions (5%) to add variance
            CASE
                WHEN UNIFORM(0, 100, RANDOM()) < 5 THEN
                    CASE bc.LOYALTY_TIER
                        WHEN 'high' THEN 1
                        WHEN 'medium' THEN 1
                        WHEN 'low' THEN 3
                        ELSE 2
                    END
                ELSE bc.TIER_SCORE
            END AS TIER_SCORE_NOISY,
            -- AVG_ORDER_VALUE with extra randomness
            ROUND(GREATEST(15, LEAST(500,
                UNIFORM(50, 150, RANDOM())
                + (bc.ANNUAL_INCOME / 5000)
                + ( (CASE
                        WHEN UNIFORM(0,100,RANDOM()) < 5 THEN -1 ELSE 1 END)
                      * (TIER_SCORE_NOISY * 20)
                  )
                + UNIFORM(-50, 50, RANDOM())
            )), 2) AS AVG_ORDER_VALUE,
            -- PURCHASE_FREQUENCY with expanded tenure-driven randomness
            ROUND(GREATEST(0.05, LEAST(10,
                0.5
                + (TIER_SCORE_NOISY * 0.8)
                + (UNIFORM(0, 300, RANDOM()) / 100.0)
                - (CASE WHEN bc.TENURE_MONTHS < 6 THEN UNIFORM(0.2, 0.8, RANDOM()) ELSE 0 END)
                + UNIFORM(-0.5, 0.5, RANDOM())
            )), 2) AS PURCHASE_FREQUENCY
        FROM base_customers bc
    ),
-- 3. DERIVED METRICS (Dependent on Base Metrics)
derived_metrics AS (
    SELECT 
        bm.CUSTOMER_ID,
        bm.AVG_ORDER_VALUE,
        bm.PURCHASE_FREQUENCY,
        -- RETURN_RATE (Dependent on Frequency)
        ROUND(GREATEST(0, LEAST(35,
            UNIFORM(2, 18, RANDOM()) + (CASE WHEN bm.PURCHASE_FREQUENCY > 4 THEN 6 ELSE 0 END) + UNIFORM(-6, 6, RANDOM())
        )), 2) AS RETURN_RATE,
        -- LAST_PURCHASE_DATE (Dependent on Frequency) with wider randomness
        DATEADD('day', -GREATEST(1, ROUND(
            30 / GREATEST(bm.PURCHASE_FREQUENCY, 0.3)
            + UNIFORM(0, 30, RANDOM())
        )), CURRENT_DATE()) AS LAST_PURCHASE_DATE,
        -- TOTAL_ORDERS (Dependent on Frequency) with added jitter
        GREATEST(1, ROUND(bm.PURCHASE_FREQUENCY * bm.TENURE_MONTHS * UNIFORM(0.8, 1.2, RANDOM()))) AS TOTAL_ORDERS,
        -- Pass through needed columns for LTV
        bm.TENURE_MONTHS,
        bm.TIER_SCORE_NOISY,
        bm.ANNUAL_INCOME,
        bm.SIGNUP_DATE
    FROM base_metrics bm
),
-- 4. FINAL CALCULATIONS (LTV depends on Rate, Update depends on Last Date)
final_calculations AS (
    SELECT
        dm.CUSTOMER_ID,
        dm.AVG_ORDER_VALUE,
        dm.PURCHASE_FREQUENCY,
        dm.RETURN_RATE,
        -- LIFETIME_VALUE with added variance and occasional inversions
        ROUND(GREATEST(50,
            (dm.AVG_ORDER_VALUE * dm.PURCHASE_FREQUENCY * dm.TENURE_MONTHS)
            * (1 - dm.RETURN_RATE / 100) * (1 + dm.TIER_SCORE_NOISY * 0.1) * (1 + dm.ANNUAL_INCOME / 500000)
            * (1 + UNIFORM(-0.25, 0.25, RANDOM()))
            * (CASE WHEN UNIFORM(0, 100, RANDOM()) < 6 THEN 0.6 ELSE 1 END)
            * (CASE WHEN UNIFORM(0, 100, RANDOM()) < 6 THEN 1.4 ELSE 1 END)
            + UNIFORM(-1200, 1200, RANDOM())
        ), 2) AS LIFETIME_VALUE,
        dm.LAST_PURCHASE_DATE,
        dm.TOTAL_ORDERS,
        dm.SIGNUP_DATE
    FROM derived_metrics dm
)
-- 5. FINAL SELECTION & INSERT
SELECT 
    CUSTOMER_ID,
    AVG_ORDER_VALUE,
    PURCHASE_FREQUENCY,
    RETURN_RATE,
    LIFETIME_VALUE,
    LAST_PURCHASE_DATE,
    TOTAL_ORDERS,
    -- UPDATED_AT (Depends on LAST_PURCHASE_DATE)
    -- Spread updates over the last 1–5 months, never before signup
    GREATEST(
        SIGNUP_DATE,
        DATEADD(
            'day',
            -UNIFORM(30, 300, RANDOM()),
            CURRENT_TIMESTAMP()
        )
    ) AS UPDATED_AT
FROM final_calculations;
''', session)

print(f'Created PURCHASE_BEHAVIOR table with {num_customers} rows')

In [ ]:
# CLV by Loyalty Tier (Correlation Check)
print("\n=== CLV by Loyalty Tier (Correlation Check) ===")
session.sql('''
SELECT 
    d.LOYALTY_TIER,
    COUNT(*) AS CUSTOMER_COUNT,
    ROUND(AVG(p.LIFETIME_VALUE), 2) AS AVG_CLV,
    ROUND(MIN(p.LIFETIME_VALUE), 2) AS MIN_CLV,
    ROUND(MAX(p.LIFETIME_VALUE), 2) AS MAX_CLV
FROM CUSTOMERS d
JOIN PURCHASE_BEHAVIOR p ON d.CUSTOMER_ID = p.CUSTOMER_ID
GROUP BY d.LOYALTY_TIER ORDER BY d.LOYALTY_TIER
''').show()

## -------------------------------------------------------------------------------------

## CLEAN UP


In [ ]:
# session.close()

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
formatted_time = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%A, %B %d, %Y %I:%M:%S %p %Z")

print(f"The last run time in Melbourne is: {formatted_time}")